# Eat Safe, Love

## Notebook Set Up

In [4]:
# Import dependencies
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo=MongoClient(port=27017)

In [6]:
# assign the uk_food database to a variable name
db=mongo['uk_food']

In [7]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [9]:
# assign the collection to a variable
establishments=db['establishments']


## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [59]:
# Find the establishments with a hygiene score of 20

query ={'scores.Hygiene': 20}
field={'BusinessName':1,'scores.Hygiene':1,'_id':0}

# Use count_documents to display the number of documents in the result
result_count=establishments.count_documents(query)
print('establishments with a hygiene score of 20: ',result_count)

# Display the first document in the results using pprint
result1=establishments.find(query,field)
for i in range(1):
    pprint(result1[i])

establishments with a hygiene score of 20:  41
{'BusinessName': 'The Chase Rest Home', 'scores': {'Hygiene': 20}}


In [60]:
# Convert the result to a Pandas DataFrame

results_df=pd.DataFrame(result1)


# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(results_df))

# Display the first 10 rows of the DataFrame

results_df.head(10)

Rows in DataFrame:  41


,BusinessName,scores
0,The Chase Rest Home,{'Hygiene': 20}
1,Brenalwood,{'Hygiene': 20}
2,Melrose Hotel,{'Hygiene': 20}
3,Seaford Pizza,{'Hygiene': 20}
4,Golden Palace,{'Hygiene': 20}
5,Ashby's Butchers,{'Hygiene': 20}
6,South Sea Express Cuisine,{'Hygiene': 20}
7,Golden Palace,{'Hygiene': 20}
8,The Tulip Tree,{'Hygiene': 20}
9,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [63]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.

query ={'LocalAuthorityName':{'$regex':'London'},'RatingValue':{'$gte':'4'}}
fields={'BusinessName':1,'LocalAuthorityName':1,'RatingValue':1,'_id':0}


# Use count_documents to display the number of documents in the result
count_london=establishments.count_documents(query)                                       
print('number of documents: ',count_london)
# # Display the first document in the results using pprint
results_london=establishments.find(query,fields)
for i in range(1):
    pprint(results_london[i])

number of documents:  34
{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': '4'}


In [64]:
# Convert the result to a Pandas DataFrame
london_df=pd.DataFrame(results_london)


# Display the number of rows in the DataFrame
london_df_count=len(london_df)
print('number of rows:',london_df_count)

# Display the first 10 rows of the DataFrame
london_df.head(10)

number of rows: 34


,BusinessName,RatingValue,LocalAuthorityName
0,Charlie's,4,City of London Corporation
1,Mv City Cruises Erasmus,5,City of London Corporation
2,Benfleet Motor Yacht Club,4,City of London Corporation
3,Coombs Catering t/a The Lock and Key,5,City of London Corporation
4,Tilbury Seafarers Centre,5,City of London Corporation
5,Mv Valulla,5,City of London Corporation
6,Froottree,AwaitingInspection,City of London Corporation
7,Tereza Joanne,5,City of London Corporation
8,Brick Lane Brews,4,City of London Corporation
9,The Nuance Group (UK) Limited,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [65]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = {'$gte': (51.490142 - degree_search) ,'$lte': (51.490142 + degree_search)} 
longitude = {'$gte': (0.08384 - degree_search) ,'$lte': (0.08384 + degree_search)}

query = {'geocode.latitude':latitude,'geocode.longitude':longitude,'RatingValue':'5'}
field={'BusinessName':1,'RatingValue':1,'scores.Hygiene':1,'_id':0}
sort =  [('scores.Hygiene',1)]
limit=5

# Print the results

near_penang= list(establishments.find(query,field).sort(sort).limit(limit))
pprint(near_penang)


[{'BusinessName': 'Volunteer', 'RatingValue': '5', 'scores': {'Hygiene': 0}},
 {'BusinessName': 'Plumstead Manor Nursery',
  'RatingValue': '5',
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Atlantic Fish Bar',
  'RatingValue': '5',
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Iceland', 'RatingValue': '5', 'scores': {'Hygiene': 0}},
 {'BusinessName': 'Howe and Co Fish and Chips - Van 17',
  'RatingValue': '5',
  'scores': {'Hygiene': 0}}]


In [66]:
# Convert result to Pandas DataFrame
near_penang_df=pd.DataFrame(near_penang)
near_penang_df

,BusinessName,RatingValue,scores
0,Volunteer,5,{'Hygiene': 0}
1,Plumstead Manor Nursery,5,{'Hygiene': 0}
2,Atlantic Fish Bar,5,{'Hygiene': 0}
3,Iceland,5,{'Hygiene': 0}
4,Howe and Co Fish and Chips - Van 17,5,{'Hygiene': 0}


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [67]:

# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_query={'$match':{'scores.Hygiene':0}}

# 2. Groups the matches by Local Authority
query={'$group':{'_id':'$LocalAuthorityName','count': { '$sum': 1 }}}

# 3. Sorts the matches from highest to lowest
sort={'$sort': {'count':-1}}

pipeline=[match_query,query,sort]

# Print the number of documents in the result
re=list(establishments.aggregate(pipeline))
        
print("Number of documents in result: ", len(re))
        
pprint(re[0:10])

Number of documents in result:  55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [68]:
# Convert the result to a Pandas DataFrame

no_hygiene_df=pd.DataFrame(re)

# Display the number of rows in the DataFrame
nh_rows=len(no_hygiene_df)
print('number of rows:',nh_rows)

# Display the first 10 rows of the DataFrame
no_hygiene_df.head(10)

number of rows: 55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
